In [2]:
!pip install openl3

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 8.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 69.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327032 sha256=a78070a890e1b4505ad6b16ad12fd7f79a85bb87088b5de1f3feb4470025bd74
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29603 sha256=587c2cb920124a054afa1ef2757e6de261a972d3d422bf480d596e07a19437a9
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Creat

In [3]:
!pip show openl3

Name: openl3
Version: 0.4.2
Summary: Deep audio and image embeddings, based on Look, Listen, and Learn approach
Home-page: https://github.com/marl/openl3
Author: Aurora Cramer, Ho-Hsiang Wu, Bea Steers, and Justin Salamon
Author-email: jtc440@nyu.edu
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: h5py, kapre, librosa, moviepy, numpy, resampy, scikit-image, scipy, soundfile, tensorflow
Required-by: 


In [4]:
!pip install torchopenl3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21870 sha256=681b64b5fb649184ab67543a226871b5d70aef7411ce57772bdeaba75eec02bf
  Stored in directory: /root/.cache/pip/wheels/b9/b2/05/f883527ffcb7f2ead5438a2c23439aa0c881eaa9a4c80256f4
Successfully built julius


In [5]:
!pip show torchopenl3

Name: torchopenl3
Version: 1.0.1
Summary: Deep audio and image embeddings, based on Look, Listen, and Learn approach Pytorch
Home-page: https://github.com/torchopenl3/torchopenl3/
Author: Humair Raj Khan and Gyanendra Das
Author-email: gyanendralucky9337@gmail.com
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: julius, librosa, nnAudio, numpy, resampy, scipy, soundfile, torch
Required-by: 


In [6]:
import os
import zipfile
import openl3
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [7]:
import torchopenl3

In [8]:
if torch.cuda.is_available():
    print("GPU is available and being used by PyTorch!")
else:
    print("GPU is NOT available. Please check your Kaggle settings.")

GPU is available and being used by PyTorch!


In [9]:
input_folder = "/kaggle/input/raw-00-01-02-filtered-files"  
embedding_dir = "/kaggle/working/embeddings"
tsv_file = "/kaggle/input/moodtheme-1/final_mood_labels.tsv"

In [10]:

df = pd.read_csv(tsv_file, sep='\t', header=None, names=['file', 'label'])
if df.iloc[0, 0] == "PATH":
    df = df.drop(index=0).reset_index(drop=True)

df['name'] = df['file'].apply(lambda x: x.split('/')[-1])  


name_to_label = dict(zip(df['name'], df['label']))



print(df)

                 file      label         name
0          48/948.mp3  energetic      948.mp3
1          50/950.mp3  energetic      950.mp3
2          51/951.mp3  energetic      951.mp3
3         66/2166.mp3  energetic     2166.mp3
4         47/6247.mp3       calm     6247.mp3
...               ...        ...          ...
11587  56/1422056.mp3      happy  1422056.mp3
11588  57/1422057.mp3      happy  1422057.mp3
11589  58/1422058.mp3  energetic  1422058.mp3
11590  59/1422059.mp3      happy  1422059.mp3
11591  60/1422060.mp3      happy  1422060.mp3

[11592 rows x 3 columns]


In [11]:
df['label'].value_counts()

label
happy        6291
energetic    3046
calm         1890
tense         365
Name: count, dtype: int64

In [12]:
!rm -rf /kaggle/working/*

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [13]:

final_data = []


for root, dirs, files in os.walk(input_folder):
    #print(root)
    for file in files:
        if file.endswith('.mp3'):
            name = file.replace('low.','') #os.path.splitext(file)[0]  
            #print("name:", name)
            if name in name_to_label:
                full_path = os.path.join(root, file)
                label = name_to_label[name]
                #print(full_path, label)
                final_data.append([full_path, label]) 


final_df = pd.DataFrame(final_data, columns=['audio_file_path', 'label'])


output_file = '/kaggle/working/audio_files_with_labels.tsv'


print(f"Final file saved to: {output_file}")


Final file saved to: /kaggle/working/audio_files_with_labels.tsv


In [14]:
final_df

,audio_file_path,label
0,/kaggle/input/raw-00-01-02-filtered-files/filt...,energetic
1,/kaggle/input/raw-00-01-02-filtered-files/filt...,happy
2,/kaggle/input/raw-00-01-02-filtered-files/filt...,tense
3,/kaggle/input/raw-00-01-02-filtered-files/filt...,happy
4,/kaggle/input/raw-00-01-02-filtered-files/filt...,calm
...,...,...
7177,/kaggle/input/raw-00-01-02-filtered-files/filt...,happy
7178,/kaggle/input/raw-00-01-02-filtered-files/filt...,energetic
7179,/kaggle/input/raw-00-01-02-filtered-files/filt...,calm
7180,/kaggle/input/raw-00-01-02-filtered-files/filt...,happy


In [15]:
final_df['label'].value_counts()

label
happy        4012
energetic    1886
calm         1098
tense         186
Name: count, dtype: int64

In [16]:
import librosa
from sklearn.preprocessing import LabelEncoder


In [17]:

df = final_df.copy()  


label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

df


,audio_file_path,label
0,/kaggle/input/raw-00-01-02-filtered-files/filt...,1
1,/kaggle/input/raw-00-01-02-filtered-files/filt...,2
2,/kaggle/input/raw-00-01-02-filtered-files/filt...,3
3,/kaggle/input/raw-00-01-02-filtered-files/filt...,2
4,/kaggle/input/raw-00-01-02-filtered-files/filt...,0
...,...,...
7177,/kaggle/input/raw-00-01-02-filtered-files/filt...,2
7178,/kaggle/input/raw-00-01-02-filtered-files/filt...,1
7179,/kaggle/input/raw-00-01-02-filtered-files/filt...,0
7180,/kaggle/input/raw-00-01-02-filtered-files/filt...,2


In [18]:
df['label'].value_counts()

label
2    4012
1    1886
0    1098
3     186
Name: count, dtype: int64

In [19]:
audio0, label0 = df['audio_file_path'].iloc[0], df['label'].iloc[0]
print(audio0, label0)

/kaggle/input/raw-00-01-02-filtered-files/filtered_files_16-20/749618.low.mp3 1


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
wave_audio0, sr0 = librosa.load(audio0, sr=16000)
print(wave_audio0, sr0)

[ 0.00010839  0.00050483  0.00070658 ... -0.00243625 -0.00268697
  0.        ] 16000


In [22]:
print(wave_audio0.shape)

(5331628,)


In [23]:
wave_audio = torch.tensor(wave_audio0, dtype=torch.float32).unsqueeze(0).to(device)  

In [24]:
input_repr = 'mel256'  
content_type = 'music'  
embedding_size = 512  

# Load the model
model = torchopenl3.models.load_audio_embedding_model(
    input_repr=input_repr,
    content_type=content_type,
    embedding_size=embedding_size
)


/opt/conda/lib/python3.10/site-packages/torchopenl3/models.py:200: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = librosa.filters.mel(
Downloading: "https://github.com/torchopenl3/torchopenl3-models/raw/master/torchopenl3_mel256_music_512.pth.tar" to /root/.cache/torch/hub/checkpoints/torchopenl3_mel256_music_512.pth.tar
100%|██████████| 34.9M/34.9M [00:00<00:00, 270MB/s]


In [25]:
model=model.to(device)
print(model)

PytorchOpenl3(
  (speclayer): CustomMelSTFT()
  (batch_normalization_1): BatchNorm2d(1, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2d_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch_normalization_2): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2d_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch_normalization_3): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2d_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch_normalization_4): BatchNorm2d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2d_4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch_normalization_5): BatchNorm2d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2d_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (batch_normalization_6): BatchNorm2d(256, eps=0.001, momentum=0.99, affine=True, track_

In [26]:
import random
from sklearn.model_selection import train_test_split

In [27]:
file_paths = df['audio_file_path'].tolist()
labels = df['label'].tolist()

shuffled_paths, shuffled_labels = zip(*random.sample(list(zip(file_paths, labels)), len(file_paths)))


train_paths, temp_paths, train_labels, temp_labels = train_test_split(shuffled_paths, shuffled_labels, test_size=0.2, random_state=42)
val_paths, test_paths, val_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)


In [28]:
print(len(train_paths), len(val_paths), len(test_paths))

5745 718 719


In [29]:
embeddings, _ = torchopenl3.get_audio_embedding(
    torch.tensor(wave_audio), 
    model=model, 
    sr=22050  
)


print(embeddings.shape)

/tmp/ipykernel_23/169508312.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(wave_audio),


torch.Size([1, 2414, 512])


In [37]:
from tqdm import tqdm
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, file_paths, labels, model, device, sr=16000):
        self.file_paths = file_paths
        self.labels = labels
        self.model = model
        self.device = device  
        self.sr = sr

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        
        audio_path = self.file_paths[idx]
        audio, _ = librosa.load(audio_path, sr=self.sr)

        
        audio_tensor = torch.tensor(audio[:self.sr * 10], dtype=torch.float32).unsqueeze(0).to(self.device)

        
        embeddings, _ = torchopenl3.get_audio_embedding(audio_tensor, model=self.model, sr=self.sr)

        
        label = torch.tensor(self.labels[idx], dtype=torch.long).to(self.device)

        return embeddings, label


In [38]:
def save_embeddings_in_batches(file_paths, labels, model, device, batch_size=32, sr=16000, save_interval=1000, output_dir=""):
    
    dataset = AudioDataset(file_paths, labels, model, device, sr)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    all_embeddings = []
    all_labels = []

    for i, (audio_embeddings, batch_labels) in enumerate(tqdm(dataloader)):
        all_embeddings.append(audio_embeddings)
        all_labels.append(batch_labels)

        
        if (i + 1) % save_interval == 0:
            embeddings_batch = torch.cat(all_embeddings, dim=0).cpu()  
            labels_batch = torch.cat(all_labels, dim=0).cpu()
            torch.save(embeddings_batch, f"{output_dir}/audio_embeddings_batch_{i + 1}.pt")
            torch.save(labels_batch, f"{output_dir}/labels_batch_{i + 1}.pt")
            all_embeddings = []  
            all_labels = []

    
    final_embeddings = torch.cat(all_embeddings, dim=0).cpu()
    final_labels = torch.cat(all_labels, dim=0).cpu()
    torch.save(final_embeddings, f"{output_dir}/audio_embeddings_final.pt")
    torch.save(final_labels, f"{output_dir}/labels_final.pt")

    return final_embeddings, final_labels


In [31]:

output_dir = '/kaggle/working/'
embeddings, final_labels = save_embeddings_in_batches(
    train_paths,
    train_labels,
    model,
    device=device,  
    batch_size=32,
    sr=16000,
    save_interval=1000,
    output_dir=output_dir
)

100%|██████████| 180/180 [1:29:59<00:00, 30.00s/it]


In [33]:
output_dir = '/kaggle/working/test_embedding'
os.makedirs(output_dir, exist_ok=True)
test_embeddings, test_labels = save_embeddings_in_batches(
    test_paths,
    test_labels,
    model,
    device=device,
    batch_size=32,
    sr=16000,
    save_interval=1000,
    output_dir=output_dir
)


100%|██████████| 23/23 [11:44<00:00, 30.62s/it]


In [39]:
output_dir = '/kaggle/working/validation_embedding'
os.makedirs(output_dir, exist_ok=True)
val_embeddings, val_labels = save_embeddings_in_batches(
    val_paths,
    val_labels,
    model,
    device=device,
    batch_size=32,
    sr=16000,
    save_interval=1000,
    output_dir=output_dir
)


100%|██████████| 23/23 [11:16<00:00, 29.41s/it]


Code to extract PANN embeddings (used to explore PANN - not utilized thouroughly)

In [ ]:
from torch.hub import load


class AudioDataset(Dataset):
    def __init__(self, file_paths, labels=None, sr=32000):
        self.file_paths = file_paths
        self.labels = labels
        self.sr = sr

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        
        audio, _ = librosa.load(audio_path, sr=self.sr)
        
        max_length = self.sr * 10  
        if len(audio) > max_length:
            audio = audio[:max_length]
        else:
            audio = np.pad(audio, (0, max_length - len(audio)), 'constant')
        
        audio_tensor = torch.tensor(audio, dtype=torch.float32)
        if self.labels is not None:
            return audio_tensor, self.labels[idx]
        return audio_tensor


def process_and_save_embeddings(file_paths, labels, model, batch_size, output_path, sr=32000):
    
    dataset = AudioDataset(file_paths, labels, sr)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    embeddings_list = []
    labels_list = [] if labels is not None else None

    os.makedirs(output_path, exist_ok=True)
    saved_batches = 0

    
    for batch in dataloader:
        audio_tensors = batch[0] if labels is not None else batch
        batch_labels = batch[1] if labels is not None else None
        
        
        with torch.no_grad():
            audio_embeddings = model(audio_tensors)  
            embeddings_list.append(audio_embeddings)
            if labels_list is not None:
                labels_list.append(batch_labels)

        
        saved_batches += 1
        batch_embeddings_path = os.path.join(output_path, f"embeddings_batch_{saved_batches}.pt")
        torch.save(audio_embeddings, batch_embeddings_path)
        print(f"Saved batch {saved_batches} embeddings to {batch_embeddings_path}")
        
    
    embeddings_tensor = torch.cat(embeddings_list, dim=0)
    torch.save(embeddings_tensor, os.path.join(output_path, "final_embeddings.pt"))
    print(f"Saved final embeddings to {os.path.join(output_path, 'final_embeddings.pt')}")
    
    if labels_list is not None:
        labels_tensor = torch.cat(labels_list, dim=0)
        torch.save(labels_tensor, os.path.join(output_path, "final_labels.pt"))
        print(f"Saved labels to {os.path.join(output_path, 'final_labels.pt')}")


def load_panns_model():
    
    model = load('qiuqiangkong/audioset_tagging_cnn', 'mobilenetv1_1k_mel', source='github')
    model.eval()  
    return model


if __name__ == "__main__":
    
    train_file_paths = train_paths
   
    
    train_labels = train_labels  
    
    
    model = load_panns_model()
    
   
    process_and_save_embeddings(train_paths, train_labels, model, batch_size=8, output_path="embeddings/train", sr=32000)
    
